In [ ]:
#save test_csv to csv
pd.read_excel(PATH_TEST).to_csv("/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/Data/data_test.csv", index=False)

#save data12152025_test to csv
pd.read_excel("/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/Data/data12152025_test.xlsx").to_csv("/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/Data/data12152025_test.csv", index=False)



#save features_with_subcat_names to csv
pd.read_excel(PATH_SUBCATEGORIA_RESULTADO).to_csv("/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/src/keras/features_with_subcat_names.csv", index=False)


#sabe item resultados as csv in my path


/home/juanchx/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/home/juanchx/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [76]:
pd.read_excel("/home/juanchx/Documents/Trabajo/recomendation_system/Sistema_recomnendacion-20250924T195047Z-1-001/Sistema_recomnendacion/Matriz/df_item_pacom_score_final_general.xlsx").to_csv("/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/src/keras/df_item_pacom_score_final_general.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
PATH_SUBCATEGORIA_RESULTADO = "/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/src/keras/features_with_subcat_names.csv"
PATH_TEST = "/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/Data/data_test.csv"
PATH_TEST_FINAL = "/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/Data/data12152025_test.csv"
PATH_ITEM_RESULTADO = "/home/juanchx/Documents/Trabajo/SYSTEM_RECOMENDATION_FNN/src/keras/df_item_pacom_score_final_general.csv"
TOP_K = 3



#familia subcategorías

def load_resultados(cod_familia: int,
                    path:str,
                    top_k:int = 3,
                    col_score:str = 'SCORE_SUBCATEGORIA',
                    col_division_comercial:bool = False) -> pd.DataFrame:
    df = pd.read_csv(path)
    df = df[df['CODIGO_FAMILIA'] == cod_familia]
    if not col_division_comercial:
        df = df.nlargest(top_k, col_score).reset_index(drop=True)
    else:
        #take top_k for each division comercial
        df = df.groupby('NOMBRE_DIVISION_COMERCIAL').apply(lambda x: x.nlargest(top_k, col_score)).reset_index(drop=True)
    print(f'Cod Familia: {cod_familia}')
    return df
def load_test(cod_familia: int,
              path:str,
              nivel : str = 'COD_SUBCATEGORIA'):
    df = pd.read_csv(path)
    categorias = df[df['CODIGO_FAMILIA'] == cod_familia][nivel].unique()
    return categorias
    
    
def metrics(df: pd.DataFrame,
            categorias: np.array,
            top_k:int = 3,
            col_division_comercial:bool = False,
            nivel : str = 'COD_SUBCATEGORIA'):
    if not col_division_comercial:
        recomendaciones = df[nivel].unique()

        recomendaciones = {'total': recomendaciones}
    else:
        recomendaciones = df.groupby('NOMBRE_DIVISION_COMERCIAL')[nivel].apply(lambda x: x.unique()).to_dict()
    
    for k,v in recomendaciones.items():
        precision = len(set(v) & set(categorias)) / top_k
        recall = len(set(v) & set(categorias)) / len(categorias) if len(categorias) > 0 else 0.0
        hit_rate = 1.0 if len(set(v) & set(categorias)) > 0 else 0.0
        print('#' * 50)
        print(f"Division Comercial: {k}")
        print(f"Precision@{top_k}: {precision:.4f} ({precision*100:.1f}%)")
        print(f"Recall@{top_k}: {recall:.4f} ({recall*100:.1f}%)")
        print(f"Hit Rate@{top_k}: {hit_rate:.4f} ({hit_rate*100:.1f}%)")

        print("\n")
        print(f'Top {top_k}: {v}')
        print(f'categorias: {categorias}')
        print(f'Número de categorias: {len(categorias)}')
        print(f'aciertos: {set(v) & set(categorias)}')
        print("\n")
    
    return precision, recall, hit_rate
        

## Busqueda de Recomendacion para una Familia especifica
*  Nivel: de subcategoria
*  Top K: 3
*  Data Test:  01-12-2025 al 09-12-2025
*  Modelo: FNN
*  Division Comercial: False

In [88]:
familia = 1719363028
resultados = load_resultados(cod_familia =  familia ,
                    path = PATH_SUBCATEGORIA_RESULTADO,
                    col_score = 'SCORE_SUBCATEGORIA',
                    col_division_comercial= False)

test = load_test(cod_familia = familia,
                path = PATH_TEST,
                nivel = 'COD_SUBCATEGORIA')

precision, recall, hit_rate = metrics(df= resultados,
            categorias= test,
            top_k= 3,
            col_division_comercial= False,
            nivel = 'COD_SUBCATEGORIA')

Cod Familia: 1719363028
##################################################
Division Comercial: total
Precision@3: 0.6667 (66.7%)
Recall@3: 0.1818 (18.2%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9633 8792 9278]
categorias: [9749 9084 2686 9369 8792 9461 9278 2362 9009 9072 9592]
aciertos: {8792, 9278}




## Busqueda de Recomendacion para una Familia especifica
*  Nivel: de subcategoria
*  Top K: 3
*  Data Test:  01-12-2025 al 09-12-2025
*  Modelo: FNN
*  Division Comercial: True

In [89]:
familia = 1719363028
resultados = load_resultados(cod_familia =  familia ,
                    path = PATH_SUBCATEGORIA_RESULTADO,
                    col_score = 'SCORE_SUBCATEGORIA',
                    col_division_comercial= True)

test = load_test(cod_familia = familia,
                path = PATH_TEST,
                nivel = 'COD_SUBCATEGORIA')

precision, recall, hit_rate = metrics(df= resultados,
            categorias= test,
            top_k= 3,
            col_division_comercial= True,
            nivel = 'COD_SUBCATEGORIA')

Cod Familia: 1719363028
##################################################
Division Comercial: CONSUMO
Precision@3: 0.3333 (33.3%)
Recall@3: 0.0909 (9.1%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9633 8792 8776]
categorias: [9749 9084 2686 9369 8792 9461 9278 2362 9009 9072 9592]
aciertos: {8792}


##################################################
Division Comercial: FARMA
Precision@3: 0.3333 (33.3%)
Recall@3: 0.0909 (9.1%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9278 9374 6846]
categorias: [9749 9084 2686 9369 8792 9461 9278 2362 9009 9072 9592]
aciertos: {9278}




/tmp/ipykernel_1740313/511116956.py:24: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('NOMBRE_DIVISION_COMERCIAL').apply(lambda x: x.nlargest(top_k, col_score)).reset_index(drop=True)


## Sin división Comercial y Subcategoria, Top 3
* Data Test:  01-12-2025 al 09-12-2025

In [ ]:
df_test = pd.read_csv(PATH_TEST)
familias = df_test['CODIGO_FAMILIA'].unique()
print(len(familias))
familias_0_aciertos = []
familias_1_aciertos = []
familias_2_aciertos = []
familias_3_aciertos = []
precision_f = 0
recall_f = 0
hit_rate_f = 0
familia_count = 1
for familia in familias:
    
    resultados = load_resultados(cod_familia =  familia ,
                        path = PATH_SUBCATEGORIA_RESULTADO,
                        col_score = 'SCORE_SUBCATEGORIA',
                        )
    test = load_test(cod_familia = familia,
                path = PATH_TEST,
                nivel = 'COD_SUBCATEGORIA')

    precision, recall, hit_rate = metrics(df= resultados,
            categorias= test,
            top_k= 3,
            col_division_comercial= False,
            nivel = 'COD_SUBCATEGORIA')
    print(f"Familia {familia_count} de {len(familias)}")
    if precision == 0:
        familias_0_aciertos.append(familia)
    elif round(precision,2) == 0.33:
        familias_1_aciertos.append(familia)
    elif round(precision,2) == 0.67:
        familias_2_aciertos.append(familia)
    elif round(precision,2) == 1:
        familias_3_aciertos.append(familia)
    precision_f += precision
    recall_f += recall
    hit_rate_f += hit_rate
    familia_count += 1
    

precision_f = precision_f/len(familias)
recall_f = recall_f/len(familias)
hit_rate_f = hit_rate_f/len(familias)
print(f"Precision: {precision_f}")
print(f"Recall: {recall_f}")
print(f"Hit Rate: {hit_rate_f}")
    
    

##################################################
Precision@3: 0.0000 (0.0%)
Recall@3: 0.0000 (0.0%)
Hit Rate@3: 0.0000 (0.0%)


Top 3: [9331 2384 6846]
categorias: [9719 9375 2686 9461 9283]
aciertos: set()


Familia 1 de 532
##################################################
Precision@3: 0.0000 (0.0%)
Recall@3: 0.0000 (0.0%)
Hit Rate@3: 0.0000 (0.0%)


Top 3: [9654 9633 9350]
categorias: [9720 9351 9400 2403 9032 9296]
aciertos: set()


Familia 2 de 532
##################################################
Precision@3: 0.3333 (33.3%)
Recall@3: 0.1667 (16.7%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9276 2384 9654]
categorias: [9098 8848 2384 9541 8832 9277]
aciertos: {2384}


Familia 3 de 532
##################################################
Precision@3: 0.3333 (33.3%)
Recall@3: 0.2000 (20.0%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [8775 8792 9694]
categorias: [6846 8777 9153 9598 9694]
aciertos: {9694}


Familia 4 de 532
##################################################
Precision@3: 0.33

In [59]:
print(f'len(familias_0_aciertos): {len(familias_0_aciertos)}')
print(f'len(familias_1_aciertos): {len(familias_1_aciertos)}')
print(f'len(familias_2_aciertos): {len(familias_2_aciertos)}')
print(f'len(familias_3_aciertos): {len(familias_3_aciertos)}')
print(f'total_familias:{len(familias_0_aciertos+familias_1_aciertos+familias_2_aciertos+familias_3_aciertos)}')



len(familias_0_aciertos): 188
len(familias_1_aciertos): 190
len(familias_2_aciertos): 113
len(familias_3_aciertos): 41
total_familias:532


In [64]:
df_test = pd.read_csv(PATH_TEST_FINAL)
familias = df_test['CODIGO_FAMILIA'].unique()
print(len(familias))
familias_0_aciertos = []
familias_1_aciertos = []
familias_2_aciertos = []
familias_3_aciertos = []
precision_f = 0
recall_f = 0
hit_rate_f = 0
familia_count = 1
for familia in familias:
    
    resultados = load_resultados(cod_familia =  familia ,
                        path = PATH_SUBCATEGORIA_RESULTADO,
                        col_score = 'SCORE_SUBCATEGORIA',
                        )
    test = load_test(cod_familia = familia,
                path = PATH_TEST_FINAL,
                nivel = 'COD_SUBCATEGORIA')

    precision, recall, hit_rate = metrics(df= resultados,
            categorias= test,
            top_k= 3,
            col_division_comercial= False,
            nivel = 'COD_SUBCATEGORIA')
    print(f"Familia {familia_count} de {len(familias)}")
    if precision == 0:
        familias_0_aciertos.append(familia)
    elif round(precision,2) == 0.33:
        familias_1_aciertos.append(familia)
    elif round(precision,2) == 0.67:
        familias_2_aciertos.append(familia)
    elif round(precision,2) == 1:
        familias_3_aciertos.append(familia)
    precision_f += precision
    recall_f += recall
    hit_rate_f += hit_rate
    familia_count += 1
    

precision_f = precision_f/len(familias)
recall_f = recall_f/len(familias)
hit_rate_f = hit_rate_f/len(familias)
print(f"Precision: {precision_f}")
print(f"Recall: {recall_f}")
print(f"Hit Rate: {hit_rate_f}")
    

581
##################################################
Precision@3: 0.0000 (0.0%)
Recall@3: 0.0000 (0.0%)
Hit Rate@3: 0.0000 (0.0%)


Top 3: [9364 6846 7044]
categorias: [9331 9318 2392]
aciertos: set()


Familia 1 de 581
##################################################
Precision@3: 0.6667 (66.7%)
Recall@3: 0.1333 (13.3%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9631 2384 8792]
categorias: [9627 9331 9631 9737 9633 9303 9294 7044 9077 6783 9121 8792 8775 9010
 9699]
aciertos: {8792, 9631}


Familia 2 de 581
##################################################
Precision@3: 0.6667 (66.7%)
Recall@3: 0.1333 (13.3%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9700 9158 9264]
categorias: [8775 2422 8777 9631 9158 9264 9633 6847 9354 8046 9295 2392 2384 6846
 8792]
aciertos: {9264, 9158}


Familia 3 de 581
##################################################
Precision@3: 0.3333 (33.3%)
Recall@3: 1.0000 (100.0%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [9633 9278 8775]
categorias: [9633]
aciertos: {9633}


F

In [63]:
print(f'len(familias_0_aciertos): {len(familias_0_aciertos)}')
print(f'len(familias_1_aciertos): {len(familias_1_aciertos)}')
print(f'len(familias_2_aciertos): {len(familias_2_aciertos)}')
print(f'len(familias_3_aciertos): {len(familias_3_aciertos)}')
print(f'total_familias:{len(familias_0_aciertos+familias_1_aciertos+familias_2_aciertos+familias_3_aciertos)}')



len(familias_0_aciertos): 137
len(familias_1_aciertos): 181
len(familias_2_aciertos): 144
len(familias_3_aciertos): 70
total_familias:532


In [70]:
#metricas por set famlias aciertos 

#media de subcategorias por familia 
print(f"media de subcategorias compradas por familia con 0 aciertos: {df_test[df_test['CODIGO_FAMILIA'].isin(familias_0_aciertos)].groupby('CODIGO_FAMILIA')['COD_SUBCATEGORIA'].nunique().mean()}")
print(f"media de subcategorias compradas por familia con 1 aciertos: {df_test[df_test['CODIGO_FAMILIA'].isin(familias_1_aciertos)].groupby('CODIGO_FAMILIA')['COD_SUBCATEGORIA'].nunique().mean()}")
print(f"media de subcategorias compradas por familia con 2 aciertos: {df_test[df_test['CODIGO_FAMILIA'].isin(familias_2_aciertos)].groupby('CODIGO_FAMILIA')['COD_SUBCATEGORIA'].nunique().mean()}")
print(f"media de subcategorias compradas por familia con 3 aciertos: {df_test[df_test['CODIGO_FAMILIA'].isin(familias_3_aciertos)].groupby('CODIGO_FAMILIA')['COD_SUBCATEGORIA'].nunique().mean()}")


media de subcategorias compradas por familia con 0 aciertos: 4.084848484848485
media de subcategorias compradas por familia con 1 aciertos: 6.194871794871795
media de subcategorias compradas por familia con 2 aciertos: 10.126666666666667
media de subcategorias compradas por familia con 3 aciertos: 12.014084507042254


In [68]:
df_test[df_test['CODIGO_FAMILIA'].isin(familias_0_aciertos)].groupby('CODIGO_FAMILIA')['COD_SUBCATEGORIA'].nunique().mean()



4.084848484848485

In [ ]:
# limitaciones del proyecto. 
# potencial que puede a ver. 
# por ejemplo 
# los nueclos no son los mejores
# afinando caracteristicas
# procesamiento de red neuronal
# agrupacion de items 
# clusterizacion de clientes ?
# posiblidades de sistemas de recomendacion
# proyecto mas uno dar persectiva a la presentacion
#---------------------------
# perfiles de clientes
# numero de ventas
'''
limitaciones
    poca informacion historica
    recursos de servidores
    red neuronal no tiene la mejor infraestructura
    limitados a ciretas variables
    informacion de nucleos
    caracteristicas que se balancean se pueden afinar  (aumentar o mejorar el concepto)
    clusterizacion de items
    clusterizacion de clientes (perfiles de clientes)
    porcentajes de sistemas de recomendacion
    ciclos de compra
'''

In [83]:
familia = 1719363028
resultados = load_resultados(cod_familia =  familia ,
                    path = PATH_ITEM_RESULTADO,
                    col_score = 'SCORE_ITEM',
                    col_division_comercial= False)

test = load_test(cod_familia = familia,
                path = PATH_TEST_FINAL,
                nivel = 'COD_ITEM')

print(f'familia: {familia}')
precision, recall, hit_rate = metrics(df= resultados,
            categorias= test,
            top_k= 3,
            col_division_comercial= False,
            nivel = 'COD_ITEM')

familia: 1719363028
##################################################
Precision@3: 0.3333 (33.3%)
Recall@3: 0.0625 (6.2%)
Hit Rate@3: 1.0000 (100.0%)


Top 3: [100160139    293004 100081331]
categorias: [100160139    573123    559929 100425538 100204344    295996 100274354
    570406     97958     63136    579650    535633 100187147    292037
    562055 100232733]
aciertos: {100160139}


